In [ ]:
s = '<option value="50">Авиа КАСКО</option><option value="76">Агрострахование</option><option value="3">Активы</option><option value="79">Входящее перестрахование от резидентов</option><option value="11">Гарантийный фонд</option><option value="43">ДСАГО</option><option value="46">Зеленая карта</option><option value="94">Инвестиции в ОВГЗ</option><option value="9">Исходящее перестрахование</option><option value="42">КАСКО</option><option value="45">Медицинское страхование (ДМС)</option><option value="88">Морское страхование</option><option value="69">Нераспределенная прибыль</option><option value="38">ОСАГО</option><option value="52">Обязательное авиастрахование</option><option value="89">Ответственность судовладельца</option><option value="83">Премии от физических лиц</option><option value="85">Премии от юридических лиц</option><option value="93">Прямое страхование</option><option value="2">Собственный капитал</option><option value="63">Страхование грузов и багажа</option><option value="48">Страхование здоровья на случай болезни</option><option value="24">Страхование имущества</option><option value="47">Страхование от несчастного случая</option><option value="64">Страхование от огневых рисков</option><option value="49">Страхование туристов (медрасходов)</option><option value="6">Страховые выплаты</option><option value="1">Страховые премии</option><option value="4">Страховые резервы</option><option value="65">Темп роста премий</option><option value="10">Уровень выплат</option><option value="80">Чистые страховые премии</option></select>'

In [ ]:
import re
import requests
import lxml.html as lh
import numpy as np
import pandas as pd

names=re.findall(r'>(.*?)<', s)
idx=re.findall(r'"(.*?)"', s)
names=[names[2*i] for i in range(len(idx))]

THEMES=dict(zip(names, idx))


NAMES=['АХА', 'АХА СТРАХОВАНИЕ', 'ARX']
YEARS=[i for i in range(14, 20)]
PERIODS=[3, 6, 9, 12]

URL='https://forinsurer.com/ratings/nonlife/{}/{}/{}'


In [ ]:
def bad_condition(a, b):
    if (a[0] in b): return int(np.where(b==a[0])[0])
    if (a[1] in b): return int(np.where(b==a[1])[0])
    if (a[2] in b): return int(np.where(b==a[2])[0])
    return False

In [ ]:
df = pd.DataFrame(columns = ['Рік', 'Період']+list(THEMES.keys()))

In [ ]:
idx=0

for year in YEARS:
    for period in PERIODS:
        
        list_of_y_p = [year, period]
        for theme in THEMES:
            
            
            html = requests.get(URL.format(year, period, THEMES[theme])).content
            df_list = pd.read_html(html,encoding='windows-1251')
            
            df_list = df_list[1]
            
            local_names = df_list[1].values
            
            rating = bad_condition(NAMES, local_names)
            
            if rating:
                list_of_y_p.append(rating)
            else:
                list_of_y_p.append(np.nan)
        
        df.loc[idx] = list_of_y_p
        idx+=1 
            
            
        

In [ ]:
df.to_csv('Ratings2.csv', index=False)

In [ ]:
df2 = pd.DataFrame(columns = ['Рік', 'Період'] + list(THEMES.keys()))

idx=0

for year in YEARS:
    for period in PERIODS:
        list_of_y_p = [year, period]
        for theme in THEMES:
            
            html = requests.get(URL.format(year, period, THEMES[theme])).content
            df_list = pd.read_html(html,encoding='windows-1251')
            
            df_list = df_list[1]
            
            local_names = df_list[1].values
            
            rating = bad_condition(NAMES, local_names)
            
            if rating:
                
                local_col_names = df_list.iloc[0][2:].values
                local_col_names = [str(ss).replace('\xad', '') for ss in local_col_names if str(ss)!='nan' ] 
                
                local_col_list = df_list.iloc[rating][2:]
                
                local_dict = dict(zip(local_col_names, local_col_list))
                
                list_of_y_p.append(local_dict)
            else:
                list_of_y_p.append(np.nan)
        
        
        df2.loc[idx] = list_of_y_p
        idx+=1 

In [ ]:
df2